In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sci
from numpy.linalg import inv

### Load the data

In [2]:
audio = sci.loadmat("amp_data.mat", squeeze_me=False)

In [ ]:
audio = np.array(audio['amp_data'])

# Q1

# a)

### Plot a line graph of data

In [ ]:
plt.plot(audio)

### Plot histogram of data

In [ ]:
bins = plt.hist(audio, 100)

- The histogram shows that the amplitudes are similar to a Normal distribution with mean 0.
- There is possibly some noise (pun intended) in the data as exhibited by extreme values.
- It has a low variance shown by having such short tails.


# b)

### Generate the matrix

In [ ]:
C = len(audio) // 21
throw_away = len(audio) % 21
audio = audio[:-throw_away]
audio = audio.reshape(C, 21)

### Randomly shuffle the rows of the matrix

In [ ]:
np.random.seed(0)
np.random.shuffle(audio)

### Split into training, test and validation sets

In [ ]:
train_size = int(np.floor(0.7 * C))
val_size = int(np.floor(0.15 * C))
test_size = int(C - train_size - val_size)

In [ ]:
train_set = audio[0:train_size]

In [ ]:
test_set = audio[train_size+1:train_size+test_size]

In [ ]:
val_set = audio[train_size+test_size+1:]

### Training

In [ ]:
X_shuf_train = train_set[:, :20]

In [ ]:
y_shuf_train = train_set[:, 20:21]

### Validation

In [ ]:
X_shuf_val = val_set[:, :20]

In [ ]:
y_shuf_val = val_set[:, 20:21]

### Test

In [ ]:
X_shuf_test = test_set[:, :20]

In [ ]:
y_shuf_test = test_set[:, 20:21]

# Q2

### a) Straight Line and Quartic Fit

In [ ]:
def phi_linear(Xin):
    return np.array([np.ones(len(Xin)), Xin]).T

def phi_poly(Xin):
    return np.array([np.ones(len(Xin)), Xin, Xin**2, Xin**3, Xin**4]).T

In [ ]:
x_ticks = np.array([round(x/float(20), 2) for x in range(0, 20)])
poly_ticks = np.append(x_ticks, [1])

row_number = np.random.randint(0, X_shuf_train.shape[0])

plt.plot(x_ticks, X_shuf_train[row_number, :], 'bx')

linear_fit = np.linalg.lstsq(phi_linear(x_ticks), X_shuf_train[row_number, :])[0]
line = np.dot(phi_linear(poly_ticks), linear_fit)
plt.plot(poly_ticks, line, 'r-')

quar_fit = np.linalg.lstsq(phi_poly(x_ticks), X_shuf_train[row_number, :])[0]
poly_line = np.dot(phi_poly(poly_ticks), quar_fit)
plt.plot(poly_ticks, poly_line, 'g-')

plt.plot([1], y_shuf_train[row_number, :], 'ro', mew=1, ms=10)

plt.show()

## b)

A linear fit on the last two points could be better because the curve is continuous and the gradient of the curve changes slowly. Therefore the gradient of the line between points (n-2) and (n-1) will be very similar to the gradient of the line between (n-1) and n (n being the point we are predicting).

Using the linear fit across all 20 points averages the gradient across all points and therefore it doesn't closely fit gradient of the line between (n-1) and n.

Inspecting the data we can see that the rows of our matrix form many different shaped curves. Therefore the quartic model is better because its gradient can alternate between positive and negative values to better fit the non-linear curves displayed by the points.



### c)

Judging by the graphs drawn above we would predict that a quartic fit with context length ~20 would be best at predicting the 21st point. A higher order polynomial might overfit the data and a lower order polynomial might not closely fit the varying amplitudes.

# Q3

### a)

In [ ]:
%%latex
Due to dimensionality analysis,
<br>
$f(t=1) = (f(t=1))^{T} = [((\pmb{\Phi}^{T}\pmb{\Phi})^{-1}\pmb{\Phi}^{T}\textbf{x})\pmb{\phi}(t=1)]^{T}$
<br>
<br>
And therefore,
<br>
$f(t=1) = \pmb{\phi}^{T}(t=1)(\pmb{\Phi}^{T}\pmb{\Phi})^{-1}\pmb{\Phi}^{T}\textbf{x}$
<br>
<br>
By comparing to $f(t=1) = \textbf{v}^{T}\textbf{x}$, we obtain,
<br>
$\textbf{v}^{T} = \pmb{\phi}^{T}(t=1)(\pmb{\Phi}^{T}\pmb{\Phi})^{-1}\pmb{\Phi}^{T}$
<br>
<br>
And consequently,
<br>
$\textbf{v} = \pmb{\Phi}(\pmb{\Phi}^{T}\pmb{\Phi})^{-T}\pmb{\phi}(t=1)$.

### b) i)

In [ ]:
def row_constructor(c, K):
    return np.array([c**i for i in range(0, K)])

def Phi(C, K):
    t_values = np.linspace((19-C+1)/float(20), 19/float(20), C)
    return np.array([row_constructor(i, K) for i in t_values])

### ii)

In [ ]:
def make_vv(C, K):
    phi_1 = row_constructor(1, K).T
    phi = Phi(C, K)
    return np.array([phi_1.T.dot(inv(phi.T.dot(phi))).dot(phi.T)])

### iii)

In [ ]:
row_number = np.random.randint(0, X_shuf_train.shape[0])
training_row = X_shuf_train[row_number, :]

### Model using matrix VV

In [ ]:
linear_vv = make_vv(20, 2)
quart_vv = make_vv(20, 5)

linear_vv_y = linear_vv.dot(training_row.T)
quart_vv_y = quart_vv.dot(training_row.T)

### Model from Q2

In [ ]:
x_ticks = np.array([round(x/float(20), 2) for x in range(0, 20)])
poly_ticks = np.append(x_ticks, [1])

linear_fit_y = np.linalg.lstsq(phi_linear(x_ticks), training_row)[0]
linear_line_y = np.dot(phi_linear(poly_ticks), linear_fit_y)

quart_fit_y = np.linalg.lstsq(phi_poly(x_ticks), training_row)[0]
quart_line_y = np.dot(phi_poly(poly_ticks), quart_fit_y)

### Predictions

In [ ]:
print "Prediction using VV for linear fit, " + str(linear_vv_y)
print "Prediction using VV for quartic fit, " + str(quart_vv_y)
print "Prediction using Q2 model for linear fit, " + str(linear_line_y[20:])
print "Prediction using Q2 model for quartic fit, " + str(quart_line_y[20:])

### c)

### i)

In [ ]:
def least_sum_of_squares(vv, rows, y_values):
    y_prediction = vv.dot(rows.T)
    return np.sum(np.power((y_prediction - y_values.T), 2))

### Evaluate K and C by Least Sum of Squares

In [ ]:
training_rows = []
random_rows = np.random.randint(X_shuf_train.shape[0], size=1000000)
training_rows = X_shuf_train[random_rows,:]
actual_y_values = y_shuf_train[random_rows, :]

min_lss = 10000000
min_C = 0
min_K = 0

for C in range(2, 20):
    for K in range(1, 21):
        #Generate our matrix for C and K
        vv = make_vv(C, K)
        lss = least_sum_of_squares(vv, training_rows[:, X_shuf_train.shape[1]-C:], actual_y_values)
        
        if (lss < min_lss):
            min_C = C
            min_K = K
            min_lss = lss

### Best settings for training set

In [ ]:
print "The best setting for C on the training set is " + str(min_C)
print "The best setting for K on the training set is " + str(min_K)

Therefore we should be using C=2 and K=2 for predicting our 21st point.

### ii)

### Evaluate Model on Training, Validation and Test Set

In [ ]:
def mean_square_error(vv, rows, y_values):
    y_prediction = vv.dot(rows.T)
    return (1/float(rows.shape[0]))*np.sum(np.power((y_prediction - y_values.T), 2))

In [ ]:
def get_mean_square_error_from_set(rows, y_values):    
    vv = make_vv(2, 2)
    
    min_mse = 10000000

    mse = mean_square_error(vv, rows[:, rows.shape[1]-2:], y_values)

    if (mse < min_mse):
        min_mse = mse
        
    return min_mse

### Train Set

In [ ]:
print "The mean square error on the train set is: " + str(get_mean_square_error_from_set(X_shuf_train, y_shuf_train))

### Validation Set

In [ ]:
print "The mean square error on the validation set is: " + str(get_mean_square_error_from_set(X_shuf_val, y_shuf_val))

### Test Set

In [ ]:
print "The mean square error on the test set is: " + str(get_mean_square_error_from_set(X_shuf_test, y_shuf_test))

# Q4

### a)

### Evaluate different values of C for linear model

In [ ]:
def test_different_c_on_mse(rows, y_values):
    min_mse = 10000000
    min_C = 0
    
    for c in range(1, 20):
        vv = make_vv(c, 2)
        mse = mean_square_error(vv, rows[:, rows.shape[1]-c:], y_values)

        if (mse < min_mse):
            min_mse = mse
            min_C = c
    return (min_mse, min_C)

##### Training Set

In [ ]:
(mse, C) = test_different_c_on_mse(X_shuf_train, y_shuf_train)
print "The mean square error of our best fitting model is " + str(mse)
print "The best fitting linear model has value C=" + str(C)

##### Validation Set

In [ ]:
(mse, C) = test_different_c_on_mse(X_shuf_val, y_shuf_val)
print "The mean square error of our best fitting model is " + str(mse)
print "The best fitting linear model has value C=" + str(C)

Why is this the case?

- The difference in the gradient of the lines between every pair of data points, changes at a slow rate. Therefore a line between point (n-2) and (n-1) will likely closely predict point n. And so the line going through the 19th and 20th point will very likely pass closely to the 21st point, giving us a low mean square error.

### b)

Comparing approaches:

- Our approach from Q3 and Q4a) resulted in the same model and therefore their mean squared errors are the same.

### c)

In [ ]:
vv = make_vv(2, 2)
y_prediction = vv.dot(X_shuf_val[:, X_shuf_val.shape[1]-2:].T)
residuals = (y_prediction - y_shuf_val.T)

In [ ]:
residual_bins = plt.hist(residuals.T, 100)

- The histogram for the mse appears almost identical to the histogram of the original amplitudes. 
- There appears to be some outlying large residuals which could be caused by sudden changes in amplitude

# Q5

- To predict point n, instead of using the interval (n-21, n-1), use the interval (n-10, n+10) omitting the point n. This could give a much better prediction of point n.
- Using regularization could help the polynomial models (K > 2) have a lower M.S.E than the linear fit (K = 2)

As exhibited in the below plot, the end behaviour of the quartic fit does not closely predict the final point of the data. Because having access to points on both sides of the 21st would allow a better fit for the 21st point.

In [ ]:
x_ticks = np.array([round(x/float(20), 2) for x in range(0, 20)])
poly_ticks = np.append(x_ticks, [1])

row_number = np.random.randint(0, X_shuf_train.shape[0])

plt.plot(x_ticks, X_shuf_train[row_number, :], 'bx')

linear_fit = np.linalg.lstsq(phi_linear(x_ticks), X_shuf_train[row_number, :])[0]
line = np.dot(phi_linear(poly_ticks), linear_fit)
plt.plot(poly_ticks, line, 'r-')

quar_fit = np.linalg.lstsq(phi_poly(x_ticks), X_shuf_train[row_number, :])[0]
poly_line = np.dot(phi_poly(poly_ticks), quar_fit)
plt.plot(poly_ticks, poly_line, 'g-')

plt.plot([1], y_shuf_train[row_number, :], 'ro', mew=1, ms=10)

plt.show()